### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('./data/movies_clean.csv')
reviews = pd.read_csv('./data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
user_movie_subset

movie_id,73486,75314,68646,99685
user_id,,,,
265,10.0,10.0,10.0,10.0
1023,10.0,4.0,9.0,10.0
1683,8.0,9.0,10.0,5.0
6571,9.0,8.0,10.0,10.0
11639,10.0,5.0,9.0,9.0
13006,6.0,4.0,10.0,6.0
14076,9.0,8.0,10.0,9.0
14725,10.0,5.0,9.0,8.0
23548,7.0,8.0,10.0,8.0


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [37]:
np.isnan(1) == False

True

In [38]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies = ratings_mat.shape[1]# number of movies in the matrix
    num_ratings = np.sum(ratings_mat[:].reshape(-1) == np.nan) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users,latent_features)# user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for idx_iter in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for idx_user in range(n_users):
            for idx_movie in range(n_movies):
                # if the rating exists
                if np.isnan(ratings_mat[idx_user,idx_movie]) == False :
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    err = ratings_mat[idx_user,idx_movie] - user_mat[idx_user,:] @ movie_mat[:,idx_movie]
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum = sse_accum + err**2
                    # update the values in each matrix in the direction of the gradient
                    user_mat[idx_user,:] = user_mat[idx_user,:] + learning_rate*2*err*movie_mat[:,idx_movie].T
                    movie_mat[:,idx_movie] = movie_mat[:,idx_movie] + learning_rate*2*err*user_mat[idx_user,:].T                    
        # print results
        print('Iteration {} | Error {}'.format(idx_iter,sse_accum))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [24]:
user_mat, movie_mat = FunkSVD(ratings_mat, 4,0.005, 10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
Iteration 0 | Error 3562.12361544858
Iteration 1 | Error 1302.9347272726218
Iteration 2 | Error 351.7270856339102
Iteration 3 | Error 223.08882268614943
Iteration 4 | Error 208.7591248189731
Iteration 5 | Error 202.96933491746455
Iteration 6 | Error 198.01403950628193
Iteration 7 | Error 192.87762152465848
Iteration 8 | Error 187.32045346750527
Iteration 9 | Error 181.2792896074746


In [25]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.5203752   8.98258691 10.21288878 10.15906302]
 [ 8.62067054  7.53546614  9.08917625  8.72664385]
 [ 7.50130675  6.91936833  9.09349687  7.24588516]
 [ 9.41490506  8.30759873  9.98570451  9.68019987]
 [ 8.59021266  7.34640453  8.37297206  9.06596682]
 [ 6.90003408  6.15162917  7.81100322  6.25101576]
 [ 9.26734057  8.1338488   9.72531448  9.05185286]
 [ 7.85422843  7.06739111  8.92730545  8.17807087]
 [ 8.10646012  7.45270854  9.68062437  8.00602142]
 [ 7.93899713  6.84416954  8.19052849  7.34255389]
 [ 8.6202441   7.77726447  9.79494056  8.56939893]
 [ 9.50591586  8.30025044  9.86740187  9.8270324 ]
 [ 9.55662063  8.25128225  9.66543471  9.04442048]
 [ 6.6483196   5.68660848  6.55120185  7.02669444]
 [ 9.64383754  8.5260564  10.43238153  9.60738716]
 [ 9.72843629  8.51783002 10.25459114  9.71029425]
 [ 8.47201866  7.3115016   8.49254833  8.31482657]
 [ 7.58650683  6.1819985   6.1835014   7.90488296]
 [ 8.18707973  7.26904064  8.85881836  8.50100571]
 [ 8.26878855  7.08175225  8.17

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [26]:
user_mat, movie_mat = FunkSVD(ratings_mat, 4,0.005, 250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
Iteration 0 | Error 3678.129452567044
Iteration 1 | Error 1410.1137010690725
Iteration 2 | Error 386.77199755273404
Iteration 3 | Error 233.03877898311782
Iteration 4 | Error 217.37937435406266
Iteration 5 | Error 213.1813448593211
Iteration 6 | Error 210.2902153463349
Iteration 7 | Error 207.4769434502088
Iteration 8 | Error 204.4573291173603
Iteration 9 | Error 201.13019301667654
Iteration 10 | Error 197.44467459708414
Iteration 11 | Error 193.37280103158537
Iteration 12 | Error 188.90539317848857
Iteration 13 | Error 184.05349800705284
Iteration 14 | Error 178.8505780575459
Iteration 15 | Error 173.35356597886997
Iteration 16 | Error 167.6416444064545
Iteration 17 | Error 161.81206134690328
Iteration 18 | Error 155.97287413781137
Iteration 19 | Error 150.23325830830936
Iteration 20 | Error 144.6927527435129
Iteration 21 | Error 139.43128017607444
Iteration 22 | Error 134.50176301823436
Iteration 23 | Error 129.92660025098584
I

Iteration 242 | Error 3.615396044630767e-07
Iteration 243 | Error 3.243671252168113e-07
Iteration 244 | Error 2.910161828056153e-07
Iteration 245 | Error 2.610939573220226e-07
Iteration 246 | Error 2.3424800023850025e-07
Iteration 247 | Error 2.1016208620586137e-07
Iteration 248 | Error 1.885524909457999e-07
Iteration 249 | Error 1.691646515321017e-07


In [27]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.999917    9.99995031  9.99997853 10.00008767]
 [ 9.99994952  3.9999703   8.99998777 10.00005408]
 [ 7.99995387  8.99997375  9.9999897   5.00004938]
 [ 8.99994785  7.99997118  9.99998927 10.00005594]
 [ 9.99993411  4.99996347  8.99998623  9.00007043]
 [ 6.00005213  4.00003255 10.00001569  5.99994716]
 [ 9.00003318  8.00002173 10.0000113   8.99996711]
 [10.00007417  5.00004469  9.00002051  7.99992436]
 [ 6.99997474  7.99998582  9.99999472  8.00002753]
 [ 8.99999589  4.99999784  8.99999975  7.00000569]
 [ 8.99998051  7.99998939  9.99999649  8.00002179]
 [ 9.00003232 10.00002027 10.00001005  8.99996781]
 [ 9.99998543  8.99999195  9.99999737  8.00001662]
 [ 4.99998204  7.99999053  4.99999687  8.0000195 ]
 [ 9.99991617  7.99995195  9.99998057 10.00008887]
 [ 9.00005109  9.00003217 10.00001571  9.99994838]
 [ 9.00009608  8.00005714  8.00002547  7.99990117]
 [10.00002233  8.00001342  1.00000623  9.99997748]
 [ 5.00002033  6.00001152 10.00000507  9.99997979]
 [ 8.00000398  7.00000125 10.00

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [28]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [39]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, 4,0.005, 250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
Iteration 0 | Error 3556.620171818358
Iteration 1 | Error 1263.2039830336228
Iteration 2 | Error 328.5831814858709
Iteration 3 | Error 216.45007990847924
Iteration 4 | Error 205.46608142730614
Iteration 5 | Error 201.2142423204322
Iteration 6 | Error 197.66843068734718
Iteration 7 | Error 193.93183657917712
Iteration 8 | Error 189.78676688791361
Iteration 9 | Error 185.16024501342153
Iteration 10 | Error 180.00776270543892
Iteration 11 | Error 174.29247635689237
Iteration 12 | Error 167.98536817908442
Iteration 13 | Error 161.06980680317176
Iteration 14 | Error 153.5472277220867
Iteration 15 | Error 145.4431653029431
Iteration 16 | Error 136.81298522040424
Iteration 17 | Error 127.7462757343607
Iteration 18 | Error 118.3684626612283
Iteration 19 | Error 108.83809690685403
Iteration 20 | Error 99.33866979287751
Iteration 21 | Error 90.06486591118589
Iteration 22 | Error 81.20471847038436
Iteration 23 | Error 72.9206928141238
Itera

Iteration 207 | Error 8.699654006092312e-09
Iteration 208 | Error 7.69927179299021e-09
Iteration 209 | Error 6.813883740116923e-09
Iteration 210 | Error 6.030276047889585e-09
Iteration 211 | Error 5.3367527328619665e-09
Iteration 212 | Error 4.722961347467262e-09
Iteration 213 | Error 4.179738703225939e-09
Iteration 214 | Error 3.698974301796781e-09
Iteration 215 | Error 3.27348944625285e-09
Iteration 216 | Error 2.896930226453983e-09
Iteration 217 | Error 2.563672792085982e-09
Iteration 218 | Error 2.2687395007204094e-09
Iteration 219 | Error 2.0077246921258382e-09
Iteration 220 | Error 1.7767289870906258e-09
Iteration 221 | Error 1.5723011287493442e-09
Iteration 222 | Error 1.3913865031262985e-09
Iteration 223 | Error 1.2312815701833041e-09
Iteration 224 | Error 1.0895935302097444e-09
Iteration 225 | Error 9.642046198839041e-10
Iteration 226 | Error 8.532405110631852e-10
Iteration 227 | Error 7.550423385718253e-10
Iteration 228 | Error 6.681419397139714e-10
Iteration 229 | Error 5.91

In [40]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.792875651671007:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [41]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(ratings_mat, 4,0.005, 20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
Iteration 0 | Error 3472.3808545514958
Iteration 1 | Error 1177.1022876702993
Iteration 2 | Error 323.0490309609951
Iteration 3 | Error 220.07283339906886
Iteration 4 | Error 210.44456469945536
Iteration 5 | Error 206.9712932428828
Iteration 6 | Error 203.92232593890822
Iteration 7 | Error 200.65375521090527
Iteration 8 | Error 197.01256512668343
Iteration 9 | Error 192.92368638863582
Iteration 10 | Error 188.3216889441845
Iteration 11 | Error 183.14159114690415
Iteration 12 | Error 177.32164912100666
Iteration 13 | Error 170.80880075234379
Iteration 14 | Error 163.5652184658771
Iteration 15 | Error 155.57571865923796
Iteration 16 | Error 146.855613147387
Iteration 17 | Error 137.45808740253904
Iteration 18 | Error 127.4796577648366
Iteration 19 | Error 117.06190319107749


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [46]:
# Replace each of the comments below with the correct values
num_ratings = np.sum(np.isnan(first_1000_users.reshape(-1))==False)# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = np.sum(np.isnan(first_1000_users.reshape(-1))==True)# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [47]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
